# Dian 24 秋招 TEST 2

## 绝对位置编码

In [36]:
import torch
import torch.nn as nn
import math
import numpy as np
import pandas as pd

In [37]:
class AbsPosEncoding(nn.Module):
    def __init__(self,max_pos,feature_dims):
        super().__init__()
        PE=torch.zeros(max_pos,feature_dims)
        pos=torch.arange(0,max_pos).unsqueeze(1).float()

        multi_term=torch.arange(0,feature_dims,2).float()
        multi_term=torch.exp(multi_term*(-math.log(10000.0)/feature_dims))

        PE[:,0::2]=torch.sin(pos*multi_term)
        PE[:,1::2]=torch.cos(pos*multi_term)

        self.register_buffer('PE',PE.unsqueeze(0))#用于将张量（如位置编码 PE）注册为模型的持久状态。这些张量不会作为模型参数进行训练，但会随模型保存和加载。

    def forward(self,x):
        return x+self.PE[:,:x.size(1)].clone().detach()

In [38]:
max_pos=10
feature_dims=4

model=AbsPosEncoding(max_pos,feature_dims)

x=torch.normal(0,1,(3,6,feature_dims))

output=model(x)

print(x)
print(model.PE)
print(output)

tensor([[[ 1.8542, -0.0790,  0.3715, -1.3207],
         [ 0.6978, -1.0920, -0.0966,  0.7480],
         [-0.4054,  0.8703, -0.0693,  0.0710],
         [-0.2366,  0.3228, -0.4089, -0.5504],
         [-0.3716, -2.5826, -1.4065, -0.6946],
         [-0.1809,  2.0480, -1.0623,  0.0879]],

        [[-0.8062,  0.4438, -0.2858, -0.2188],
         [ 0.4094,  0.3252, -0.3034, -0.3783],
         [ 0.0734, -1.3291, -0.7186, -0.9952],
         [-0.1928,  0.0046, -0.4383,  1.1125],
         [-1.3638, -1.9946,  1.5548, -0.2232],
         [ 0.1086,  0.9967, -0.3351, -0.2772]],

        [[-0.1189,  1.1794,  1.3195,  1.4304],
         [ 0.1944, -0.2712,  0.4383,  0.8753],
         [ 0.5415, -0.2789,  0.1481,  0.2461],
         [ 0.0921, -0.3438, -0.6447,  0.5229],
         [-0.8503, -0.3165, -1.9022,  0.7332],
         [-1.4486,  1.4166, -1.1654, -0.5996]]])
tensor([[[ 0.0000,  1.0000,  0.0000,  1.0000],
         [ 0.8415,  0.5403,  0.0100,  0.9999],
         [ 0.9093, -0.4161,  0.0200,  0.9998],
       

## 旋转位置编码

In [39]:
import torch
import torch.nn as nn
import math

class RotaryPosEncoding(nn.Module):
    def __init__(self, max_pos, feature_dims):
        super().__init__()
        self.max_pos = max_pos
        self.feature_dims = feature_dims

        # 创建旋转位置编码矩阵
        self.create_rotary_pos_encoding()

    def create_rotary_pos_encoding(self):
        # 位置张量
        position = torch.arange(0, self.max_pos).unsqueeze(1).float()
        print(position.shape)
        # 维度张量
        dim = torch.arange(0, self.feature_dims // 2).float()
        print(f'dims:{dim.shape}')
        
        # 计算角度
        angles = position*torch.exp(-math.log(10000.0) * dim / (self.feature_dims // 2))#theta
        angles = angles.unsqueeze(2)  # (max_pos, feature_dims / 2, 1)
        print(f'angles:{angles.shape}')

        # 计算旋转的正弦和余弦
        self.sin_angles = torch.sin(angles)
        print(self.sin_angles.shape)
        self.cos_angles = torch.cos(angles)
    
    def get_pos_encoding(self, seq_len):
        # 获取前seq_len个位置编码
        # 计算每个位置的sin和cos
        sin_pos = self.sin_angles[:seq_len, :, 0]  # (seq_len, feature_dims / 2)
        print(sin_pos.shape)
        cos_pos = self.cos_angles[:seq_len, :, 0]  # (seq_len, feature_dims / 2)
        
        # 组合sin和cos成完整的编码，将两者分别对自己concat,实现重复效果
        cos_pos_encoding = torch.cat([cos_pos, cos_pos], dim=1)
        print(f'a{cos_pos_encoding.shape}')
        cos=cos_pos_encoding[6,:]
        print(f's:{cos.shape}')
        cos=cos.numpy()
        cos=pd.DataFrame(cos) 
        cos.to_excel('cos_pos_encoding.xlsx') 


        sin_pos_encoding = torch.cat([sin_pos, sin_pos], dim=1)
        sin=sin_pos_encoding[6,:]
        print(f's:{sin.shape}')
        sin=sin.numpy()
        sin=pd.DataFrame(sin) 
        sin.to_excel('sin_pos_encoding.xlsx')

        print(f'11{cos_pos_encoding.shape}')
        print(f'11{sin_pos_encoding.shape}')

        return cos_pos_encoding.unsqueeze(0),cos_pos_encoding.unsqueeze(0) # (1, seq_len, feature_dims)


    def forward(self, x):#自动执行
        # x的形状是 (batch_size, seq_len, feature_dims)
        batch_size, seq_len, feature_dims = x.size()
        
        # 生成正弦和余弦位置编码
        cos_pos_encoding,sin_pos_encoding = self.get_pos_encoding(seq_len)
        
        # 将位置编码应用到x上
        x = x * cos_pos_encoding+x*sin_pos_encoding
        return x

    
# 示例用法
max_pos = 512
feature_dims = 64
x = torch.randn(32, 50, feature_dims)  # (batch_size, seq_len, feature_dims)

rop = RotaryPosEncoding(max_pos, feature_dims)
encoded_x = rop(x)
print(encoded_x.shape)  
print(encoded_x)


torch.Size([512, 1])
dims:torch.Size([32])
angles:torch.Size([512, 32, 1])
torch.Size([512, 32, 1])
torch.Size([50, 32])
atorch.Size([50, 64])
s:torch.Size([64])
s:torch.Size([64])
11torch.Size([50, 64])
11torch.Size([50, 64])
torch.Size([32, 50, 64])
tensor([[[ 3.3703, -0.0894,  1.5173,  ..., -2.4980, -1.1261, -0.9834],
         [ 0.7898,  1.0664, -1.6102,  ..., -1.1337,  0.7493,  1.9031],
         [ 0.0513, -0.0816, -0.9557,  ..., -1.2478,  0.6133,  1.2436],
         ...,
         [-2.2580,  0.6525, -0.6725,  ..., -1.9854, -1.2191,  3.9543],
         [-0.9416, -0.4037, -0.4872,  ...,  0.5792,  2.8435,  2.7197],
         [-0.1767,  1.0492, -0.3057,  ...,  1.1203, -0.1899, -0.9896]],

        [[-0.0162, -1.3850, -1.5298,  ...,  0.6609, -1.6867, -0.7495],
         [-0.0843,  0.7929, -0.8734,  ...,  1.3991, -0.3363, -3.1567],
         [-0.9206,  0.1070,  0.4149,  ..., -0.7691,  2.5169, -0.6010],
         ...,
         [ 2.1724,  1.3049, -0.1233,  ...,  2.7875, -3.4522,  2.0544],
        